In [125]:
import pandas as pd
import numpy as np
culture = pd.read_csv('서울시 문화공간 정보.csv',encoding='cp949')
culture.head(3)

,번호,주제분류,문화시설명,주소,위도,경도,전화번호,팩스번호,홈페이지,관람시간,...,기타사항,시설소개,무료구분,지하철,버스정거장,YELLOW,GREEN,BLUE,RED,공항버스
0,1,기타,경희대학교,서울특별시 동대문구 경희대로 26,37.593949,127.054891,02) 961-0114,NaN,https://www.khu.ac.kr/kor/main/index.do,NaN,...,NaN,<div style='text-align: center;' align='center...,무료,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,기타,DDP(동대문디자인플라자),서울특별시 중구 을지로 281,37.566813,127.009626,02-2153-0000,02-2153-0010,http://www.ddp.or.kr,NaN,...,NaN,<h5 class='tit_chapter' style='padding: 0px; m...,무료,NaN,"동대문역사문화공원, 광희동, 을지로 6가 국립중앙의료원",NaN,NaN,"N13, 105,152,301,N13,N16,N30,105,144,261,301,4...",NaN,NaN
2,3,공연장,우란2경,서울특별시 성동구 연무장7길 11,37.543790,127.054417,070-4244-3812,NaN,http://www.wooranfdn.org/main.jsp,NaN,...,NaN,<h3 class='space-content-title'><span style='f...,무료,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
culture.loc[culture['문화시설명'] == '주한영국문화원']

,번호,주제분류,문화시설명,주소,위도,경도,전화번호,팩스번호,홈페이지,관람시간,...,기타사항,시설소개,무료구분,지하철,버스정거장,YELLOW,GREEN,BLUE,RED,공항버스
11,12,문화원,주한영국문화원,서울특별시 중구 서소문로11길 19,37.563788,126.972089,1522-5009,02 3702 0660,https://www.britishcouncil.kr/,NaN,...,NaN,<span style='font-size: 12pt;'>영국문화원은 교육과 문화 교...,유료,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
drop_cols = ['기타사항','관람시간','번호','팩스번호','지하철','버스정거장','YELLOW','GREEN','BLUE','RED','공항버스','개관일자','객석수']
culture = culture.drop(drop_cols,axis=1)

In [128]:
culture.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012 entries, 0 to 1011
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   주제분류    1012 non-null   object 
 1   문화시설명   1012 non-null   object 
 2   주소      1011 non-null   object 
 3   위도      1011 non-null   float64
 4   경도      1011 non-null   float64
 5   전화번호    1012 non-null   object 
 6   홈페이지    1010 non-null   object 
 7   관람료     441 non-null    object 
 8   휴관일     1004 non-null   object 
 9   대표이미지   1012 non-null   object 
 10  시설소개    1007 non-null   object 
 11  무료구분    1005 non-null   object 
dtypes: float64(2), object(10)
memory usage: 95.0+ KB


In [129]:
culture['주제분류'].unique()

array(['기타', '공연장', '도서관', '문화예술회관', '미술관', '박물관/기념관', '문화원'],
      dtype=object)

In [130]:
#culture = culture.sort_values(by='주제분류').reset_index(drop=True)

## 주소 전처리

In [131]:
culture.loc[culture['주소'].isna()]
#결측치 검색해서 처리
culture['주소'] = culture['주소'].fillna('서울특별시 용산구 백범로 329')
# 앞,뒤 공백 제거
culture['주소'] = culture['주소'].str.strip()
# 주소가 서울 아님
culture_not_seoul = culture.loc[~culture['주소'].str.startswith('서울')]

# 경기도가 들어가있는 행 제거
culture = culture.loc[~culture['주소'].str.contains('경기도|경기')]


# 주소 잘못 기재된거 처리
fix = culture['문화시설명'] == '갤러리 히든플레이스'
culture.loc[fix,'주소'] = culture.loc[fix,'주소'].replace('울특별시 서대문구 대현동 53-9','서울특별시 서대문구 이화여대2가길 24 2층')
fix2 = culture['문화시설명'] == '코리아나 화장박물관'
culture.loc[fix2,'주소'] = culture.loc[fix2,'주소'].replace('강남구 언주로 827','서울 강남구 언주로 827')
fix3 = culture['문화시설명'] == '용산도시기억전시관'
culture.loc[fix3,'주소'] = culture.loc[fix3,'주소'].replace('한강로3가 64-1 , 용산센트럴파크 해링턴스퀘어 공공시설동 1,2층','서울특별시 용산구 서빙고로 17 용산센트럴파크 해링턴스퀘어 공공시설동 1,2층')

#서울 자유시민 대학은 정보가 서로 불일치 해서 드롭
culture = culture.loc[~(culture['문화시설명'] == '서울자유시민대학')]

## 위도, 경도 전처리

In [132]:
# 결측치 채우기
culture['위도'] = culture['위도'].fillna('37.5122679320541').astype('float')
culture['경도'] = culture['경도'].fillna('126.939428790239').astype('float')
# 앞.뒤 공백제거 
culture['위도'] = culture['위도'].astype('str').str.strip().astype('float')
culture['경도'] = culture['경도'].astype('str').str.strip().astype('float')

## 전화번호 전처리

In [133]:
culture.loc[culture['전화번호'] == ' ']

fix_num1 = culture['문화시설명'] == '크리에이터스 스페이스'    
culture.loc[fix_num1,'전화번호'] = culture.loc[fix_num1,'전화번호'].replace(' ','02-923-0901')
fix_num2 = culture['문화시설명'] == '동숭무대'    
culture.loc[fix_num2,'전화번호'] = culture.loc[fix_num2,'전화번호'].replace(' ','0507-1360-2695')
fix_num3 = culture['문화시설명'] == '갤러리 히든플레이스'
culture.loc[fix_num3,'전화번호'] = culture.loc[fix_num3,'전화번호'].replace(' ','0507-1393-7755')
fix_num4 = culture['문화시설명'] == '카페프로젝트이응'
culture.loc[fix_num4,'전화번호'] = culture.loc[fix_num4,'전화번호'].replace(' ','0507-1315-0284')
fix_num5 = culture['문화시설명'] == '국사봉숲속작은도서관'
culture.loc[fix_num5,'전화번호'] = culture.loc[fix_num5,'전화번호'].replace(' ','010-5849-2327')
fix_num6 = culture['문화시설명'] == '알비 (ARBY)'
culture.loc[fix_num6,'전화번호'] = culture.loc[fix_num6,'전화번호'].replace(' ','010-4127-9593')
fix_num7 = culture['문화시설명'] == 'YK Presents'
culture.loc[fix_num7,'전화번호'] = culture.loc[fix_num7,'전화번호'].replace(' ','010-6729-0405')
fix_num8 = culture['문화시설명'] == '케이움 갤러리'
culture.loc[fix_num8,'전화번호'] = culture.loc[fix_num8,'전화번호'].replace(' ','0507-1376-3050')
fix_num9 = culture['문화시설명'] == '킵인터치'
culture.loc[fix_num9,'전화번호'] = culture.loc[fix_num9,'전화번호'].replace(' ','010-9133-3209')
fix_num10 = culture['문화시설명'] == '갤러리240'
culture.loc[fix_num10,'전화번호'] = culture.loc[fix_num10,'전화번호'].replace(' ','0507-1396-2533')

## 홈페이지 전처리

In [134]:
culture.loc[culture['홈페이지'].isna()]
fix_home1 = culture['문화시설명'] == '이마트은평점'
culture.loc[fix_home1,'홈페이지'] = culture.loc[fix_home1,'홈페이지'].fillna('https://store.emart.com/main/main.do')
# 사이트 없음
fix_home2 = culture['문화시설명'] == '롯데화랑(본점)'
culture.loc[fix_home2,'홈페이지'] = culture.loc[fix_home2,'홈페이지'].fillna(' ')

### 무료구분 컬럼 채움

In [135]:
culture.loc[culture['무료구분'].isna()]
culture['무료구분'] = culture['무료구분'].fillna('무료')

## 관람료 전처리


In [136]:
# 무료구분 컬럼값이 유로이면 프로그램별 상이 추가 
culture.loc[culture['관람료'].isna()]

culture['관람료'] = culture.apply(lambda x: '무료' if pd.isna(x['관람료']) and x['무료구분'] == '무료' \
                                        else ('프로그램별 상이' if pd.isna(x['관람료']) and x['무료구분'] == '유료'\
                                             else x['관람료']), axis=1)

culture['관람료'] = culture['관람료'].apply(lambda x : '무료' if x == '0' else x)

## 휴관일 전처리 

In [137]:
culture.loc[culture['휴관일'].isna()]
def holiday(x,tar):
    if pd.isna(x):
        return tar
    return x

fix_hol1 = culture['문화시설명'] == '용산꿈나무종합타운'
culture.loc[fix_hol1,'휴관일'] = culture.loc[fix_hol1,'휴관일'].apply(lambda x:holiday(x,'매주 월요일'))

fix_hol2 = culture['문화시설명'] == '동작구 문화관광'
culture.loc[fix_hol2,'휴관일'] = culture.loc[fix_hol2,'휴관일'].apply(lambda x:holiday(x,'없음'))

fix_hol3 = culture['문화시설명'] == '서울로 7017'
culture.loc[fix_hol3,'휴관일'] = culture.loc[fix_hol3,'휴관일'].apply(lambda x:holiday(x,'연중무휴'))

fix_hol4 = culture['문화시설명'] == '장지공방'
culture.loc[fix_hol4,'휴관일'] = culture.loc[fix_hol4,'휴관일'].apply(lambda x:holiday(x,'정보없음'))

fix_hol5 = culture['문화시설명'] == '관악문화관'
culture.loc[fix_hol5,'휴관일'] = culture.loc[fix_hol5,'휴관일'].apply(lambda x:holiday(x,'정보없음'))

fix_hol6 = culture['문화시설명'] == '이마트은평점'
culture.loc[fix_hol6,'휴관일'] = culture.loc[fix_hol6,'휴관일'].apply(lambda x:holiday(x,'매월 2,4째주 일요일'))

In [138]:
# 839 인덱스 짤 - 구슬모아당구장
culture = culture.loc[~(culture['문화시설명'] == '구슬모아당구장')]

In [139]:
# 중복 행 제거 
culture.drop(index=10,inplace=True)

## 시설소개 컬럼 html요소 제거

In [140]:
from bs4 import BeautifulSoup
culture.loc[culture['시설소개'].isna()]
culture['시설소개'] = culture['시설소개'].fillna('-')
culture['시설소개'].str.strip()

def remove_html_tag(x):
    soup = BeautifulSoup(x,'html.parser')
    return soup.get_text()

culture['시설소개'] = culture['시설소개'].apply(remove_html_tag)

C:\Users\User\AppData\Local\Temp\ipykernel_25684\2009206959.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(x,'html.parser')


In [141]:
culture = culture.sort_values(by='주제분류').reset_index(drop=True)

In [142]:
culture['주제분류'].unique()

array(['공연장', '기타', '도서관', '문화예술회관', '문화원', '미술관', '박물관/기념관'],
      dtype=object)

## 공원, 놀이터  드롭

In [143]:
culture = culture.loc[~(culture['문화시설명'].str.contains('공원|놀이터'))]

## 시설소개의 단어 기준으로 실내/실외 컬럼 추가

In [144]:
def in_or_out(x):
    out_door = ['광장','야외','공터','캠핑장','정원', '공원','타종','섬']
    
    for out in out_door:
        if out in x:
            return '실외'
    return '실내'
culture['실내/실외'] = '추가'
locate = culture['주제분류'] == '기타'
culture.loc[locate,'실내/실외'] = culture.loc[locate,'시설소개'].map(in_or_out)

add = culture['실내/실외'] == '추가'
culture.loc[add,'실내/실외'] = culture.loc[add,'실내/실외'].str.replace('추가','실내')

In [145]:
culture_concert = culture.loc[culture['주제분류'] == '공연장']
culture_library = culture.loc[culture['주제분류'] == '도서관']
culture_Art_center = culture.loc[culture['주제분류'] == '문화예술회관']
culture_cultural_center = culture.loc[culture['주제분류'] == '문화원']
culture_art_museum = culture.loc[culture['주제분류'] == '미술관']
culture_museum = culture.loc[culture['주제분류'] == '박물관/기념관']
culture_rest = culture.loc[culture['주제분류'] == '기타']

In [153]:
"""
기타는 나중에 건들기로 
"""
#culture_rest['문화시설명'].unique()

'\n기타는 나중에 건들기로 \n'

In [154]:
pd.set_option('display.max_rows',None)

In [155]:
#culture_concert[['문화시설명','주소']]

## Naver 맵 crawling통해 주소 + 시설명 검색 안되는 시설 필터링

In [156]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os
import requests
from PIL import Image
from io import BytesIO
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [284]:
culture_kids = culture.loc[~(culture['주제분류'] == '기타')]

In [1]:
# 네이버 길찾기에 키워드 넣어서 가져오기 
driver = webdriver.Chrome()

output_dir = 'culture_images'
os.makedirs(output_dir,exist_ok = True)


def search_and_click(query):
    # 네이버 검색창에서 query에 해당하는 주소로 들어가기
    url = "https://map.naver.com/v5/search"
    driver.get(url)
    time.sleep(2)  
    
    try:   
        # 검색창에 쿼리 입력
        search_box = driver.find_element(By.CSS_SELECTOR,'input.input_search')
        search_box.send_keys(query)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)  # 검색 결과 로딩 대기
        
         # iframe 로드 및 전환
        # iframe으로 감싸져 있는 html페이지일 경우 그 안에 소스들을 못가져옴 
        # iframe을 변환후 다음 작업 진행 
        iframe_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "entryIframe"))
    )
        driver.switch_to.frame(iframe_element)
        
    
        # 가져오고자 하는 이미지 클래스 
        image_element = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "img.K0PDV._div,img.K0PDV"))
        )
        image_url = [img.get_attribute('src') for img in image_element]
        
        if image_url:
            return image_url[0]
        else:
            return None
    
    except Exception as e:
        print(f"이미지를 찾을 수 없습니다: {query} - {e}")
        return None


for index,row in culture_test.iterrows():
    shop_name = row['문화시설명']
    Id = row['주제분류']
    address_column = '주소'
    shop_column = '문화시설명'
    query = f"{row[address_column]} {row[shop_column]}"
    image_url = search_and_click(query)

    if image_url:
        try:
            response = requests.get(image_url)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))
            img.save(os.path.join(output_dir, f"{Id} {shop_name}.jpg"))
            print(f"Saved image for {Id} {shop_name}")
        except Exception as e:
            print(f"이미지 저장 오류: {e}")

print("Image crawling and saving completed.")
driver.quit()

In [ ]:
"""
서울특별시 중구 손기정로 101-4 손기정문화체육센터 - 삭제
서울 서초구 방배동 981-13 백석아트홀 - 방배동 981-10(주소수정)
서울특별시 종로구 평창문화로 70 서울아트센터 도암홀 - 서울아트센터 도암홀 -> 서울아트센터(이름변경)
서울  강남구 대치동  영동대로 416, 코스모타워 3층 KT&G상상마당 대치 - 서울 강남구 영동대로 416(주소변경) KT&G상상마당 대치
서울 은평구 수색로 280 (수색동) 수색동 자치회관 수색동 주민센터(다목적실) - 서울 은평구 수색로 280(주소변경) 수색동 주민센터(이름변경)
서울특별시 은평구 통일로 684 서울특별시 청년허브 - 삭제
서울특별시 강동구 성내로6길 16 (성내동) 강동어린이회관아이누리홀 - 서울 강동구 성내로6길 16 강동어린이회관(주소변경) 강동어린이회관 강동구육아종합지원센터(이름변경
서울시 강남구 일원로 90 밀알학교 세라믹 팔레스홀 - 확인필요
서울특별시 중구 세종대로19길 16 세실극장 - 국립정동극장 세실(이름변경)
서울특별시 강남구 역삼로7길 16, 3층 강남씨어터 - 서울특별시 강남구 역삼로7길 16(주소변경) 강남씨어터
서울특별시 종로구 대학로 132 아티스탄홀 - 삭제
서울특별시 성동구 서울숲길 36, 3F Studio301 STUDIO301 - 서울 성동구 서울숲길 36(주소변경) 스튜디오301(이름변경)
서울특별시 강서구 화곡로63길 65 B1 스카이아트홀 - 주소에서 B1빼기
서울특별시 서초구 강남대로 201 서초문화예술회관 (구 서초구민회관) - 서초문화예술회관(이름변경)
서울특별시 성북구 길음로7길 20 서울성북미디어문화마루 꿈빛극장 - 서울 성북구 길음로7길 20 꿈빛극장 ( 이름이든 주소든 서울성북미디어문화마루 빼기)
서울특별시 양천구 목동동로 81 양천구 해누리타운 아트홀 - 삭제
서울 강남구 봉은사로 524 코엑스인터콘티넨탈서울 코엑스 오디토리움 - 코엑스인터콘치넨탈서울 빼기
서울 강남구 학동로 171 삼익악기빌딩 3층 엠팟홀 - 주소에서 삼익악기빌딩 3층 빼기
서울특별시 성북구 아리랑로 82 아리랑 씨네센터 - 아리랑시네센터(이름변경)
서울특별시 중구 퇴계로36길 10 한국의 집 민속극장 - 서울 중구 퇴계로36길 10 한국의집 한국의집 - 주소든 이름이든 
서울특별시 종로구 인왕산로1길 21 광화문아트홀 - 삭제
서울특별시 강북구 월계로 173 북서울꿈의숲아트센터 퍼포먼스홀  - 서울 강북구 월계로 173(주소) 꿈의숲 아트센터 퍼포먼스홀(이름)
서울특별시 종로구 동숭길 130-5 B1 예그린씨어터 - 주소에서 B1빼기
서울특별시 종로구 대학로 120 (동숭동) 지하 1층 공간아울 - (동숭동) 지하 1층 - 주소에서 빼기
서울특별시 강남구 자곡로7길 3 강남구립못골한옥어린이도서관 - 못골한옥어린이도서관(이름변경)
서울  강남구 개포동  선릉로4길 30 개포도서관 - 서울특별시교육청 개포도서관(이름변경)
서울시 송파구 오금로1 (신천동) 신천빗물펌프장 4~5층 송파어린이영어도서관 - 서울 송파구 오금로 1(주소변경) 송파어린이영어도서관
서울특별시 구로구 경인로 318-15 2, 3층 구로구립개봉어린이도서관 - 삭제
서울특별시 노원구 중계로8길 20 불암도서관 - 서울 노원구 중계로 106 불암도서관(주소변경) 불암도서관
서울특별시 강서구 금낭화로24길 5 강서구립길꽃어린이도서관 - 길꽃어린이도서관(이름변경)
서울시 은평구 증산로 17길 50 내를건너서숲으로도서관 내를 건너서 숲으로 도서관 - 서울 은평구 증산로17길 50(주소변경)
서울특별시 강남구 일원로 115 삼성생명빌딩 B동 203호 강남구립열린도서관 - 열린도서관(이름변경)
서울 마포구 홍익로2길 16 (서교동, 마포평생학습관) 마포평생학습관 마포리움 - 서울 마포구 홍익로2길 16 마포평생학습관(주소) 서울특별시교육청 마포평생학습관(이름)
서울특별시 동대문구 전농로16길 97 배봉산 숲속 도서관 - 배봉산근린공원숲속도서관(이름변경)
서울특별시 서초구 나루터로 38 서초구립잠원도서관 - 잠원도서관(이름변경)
서울특별시 양천구 목동서로 113 양천도서관 - 서울특별시교육청 양천도서관(이름변경)
서울특별시 서대문구 모래내로 412 서대문도서관 - 서울특별시교육청서대문도서관(이름변경)
서울 종로구 북촌로5길 48 정독도서관 정독도서관 - 서울특별시교육청 정독도서관(이름변경)
서울특별시 서초구 반포대로 201 국립중앙도서관 실감서재 - 국립중앙도서관(이름변경)
서울특별시 서초구 반포대로 201 국립중앙도서관 실감서재 - 국립중앙도서관(이름변경)
서울특별시 서대문구 독립문공원길 80 이진아기념도서관 - 서대문구립이진아기념도서관(이름변경)
서울특별시 강서구 등촌로51나길 29 강서도서관 - 서울특별시교육청 강서도서관(이름변경)
서울특별시 동대문구 천호대로4길 22 동대문도서관 - 서울특별시교육청 동대문도서관(이름변경)
서울특별시 송파구 동남로 263 송파도서관 - 서울특별시교육청 송파도서관(이름변경)
서울특별시 구로구 고척로45길 31 고척도서관 - 서울특별시교육청 고척도서관(이름변경)
서울특별시 종로구 율곡로 6 아트코리아랩 6층 아고라 - 삭제
서울특별시 종로구 동숭길 122, 대학로센터(2, 5층) 서울문화재단 대학로센터(예술청) - 서울문화재단 대학로센터(이름변경)
서울특별시 종로구 효자로 33 아트스페이스 보안1942(통의동 보안여관) - 아트스페이스부터 이름임 -> 통의동 보안여관(이름변경)
서울특별시 마포구 양화로6길 50 공간 630 - 삭제
서울특별시 영등포구 영중로 15 영등포문화라운지 따따따 - 서울 영등포구 문래로 196 영등포구 관광정보센터(주소변경) 영등포문화라운지 따따따
서울 중구 청계천로 24 케이스퀘어빌딩 지하 1층 더 케이브 서울 - 더 케이브(이름변경)
서울  관악구 신림동 151-858  ) 관악문화관 - 삭제
서울 종로구 대학로12길 31 자유빌딩 2층 서울장애예술창작센터 - 서울문화재단 서울장애예술창작센터(이름변경)
서울특별시 송파구 올림픽로 25 잠실종합운동장 - 삭제
서울특별시 종로구 율곡로5 (사간동 122-4) 종로문화원 - 서울 종로구 율곡로 5 (주소변경)
서울특별시 은평구 은평로 111 이마트은평점 - 삭제
서울특별시 마포구 월드컵로 240 서울월드컵경기장 - 삭제
서울특별시 중구 동호로 241 장충체육관 - 삭제
서울특별시 종로구 삼일대로 467 서울노인복지센터(탑골미술관) - 탑골미술관(이름변경) 
서울특별시 강남구 도산대로67길 27 /장디자인아트 JeanDesignArt - 장디자인아트(이름변경)
서울특별시 종로구 북촌로5길 56-7 CN갤러리(정독도서관 앞) /CN갤러리 (충남문화예술 서울전시장) - CN갤러리(이름변경)
서울특별시 성북구 대사관로13길 66 뮤지엄웨이브 (구 옛돌 박물관) - 뮤지엄웨이브(이름변경)
서울특별시 강남구 논현로 745 갤러리 JJ - 서울 강남구 압구정로30길 63 (주소변경)
서울특별시 중구 세종대로 99 국립현대미술관 - 국립현대미술관 덕수궁(이름변경)
서울특별시 종로구 인사동길 34-1 인사아트프라자갤러리 - 인사아트프라자 갤러리(이름변경)
서울특별시 성동구 서울숲6길 8-1 3층10 알비 - 서울특별시 성동구 서울숲6길 8-1(주소변경)
서울특별시 노원구 화랑로 608 경춘선숲길 갤러리 - 서울 노원구 공릉동 29-1(주소변경) 경춘선숲길갤러리(이름변경)
서울특별시 종로구 인사동길 30-13 갤러리 모나리자 산촌 - 갤러리모나리자산촌(이름변경)
서울  중구 소공동 1 ) 롯데화랑(본점) - 삭제
서울 종로구 삼청로 87 도올빌딩 2층 갤러리 도올 - 서울 종로구 삼청로 87(주소변경) 갤러리 도올
서울특별시 중구 세종대로 99 국립현대미술관 덕수궁 - 주소 중복
서울특별시 종로구 인사동길 57-1 갤러리 가이아 - 삭제
서울특별시 종로구 인사동11길 6 토포하우스 - 토포하우스 서울(이름변경)
서울특별시 강남구 영동대로 513 (삼성동, 코엑스) 06164 코엑스전시장 - 삭제
서울특별시 강남구 삼성로 773 올리비아 박갤러리 - 올리비아박 갤러리(이름변경)
서울특별시 용산구 녹사평대로46길 59 갤러리필로소피 - 삭제
서울특별시 마포구 월드컵북로 396 디지털 파빌리온 - 디지털파빌리온(이름변경)
"""

## 구글 맵으로 검색 안되는 주소, 시설명 한번더 필터링

In [126]:
# 구글 길찾기에 키워드 넣어서 가져오기 
driver = webdriver.Chrome()

output_dir = 'culture_images'
os.makedirs(output_dir,exist_ok = True)

failed_query = 'failed_query.txt'

def search_and_click(query):
    # 네이버 검색창에서 query에 해당하는 주소로 들어가기
    url = "https://www.google.co.kr/maps/?hl=ko&entry=ttu"
    driver.get(url)
    time.sleep(2)  
    
    try:   
        # 검색창에 쿼리 입력
        search_box = driver.find_element(By.CSS_SELECTOR,'input.fontBodyMedium.searchboxinput.xiQnY')
        search_box.send_keys(query)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)  # 검색 결과 로딩 대기
        
         # iframe 로드 및 전환
        # iframe으로 감싸져 있는 html페이지일 경우 그 안에 소스들을 못가져옴 
        # iframe을 변환후 다음 작업 진행 
#         iframe_element = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID, "entryIframe"))
#     )
#         driver.switch_to.frame(iframe_element)
        
    
        # 가져오고자 하는 이미지 클래스 
        image_element = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'img[src^="https://lh5.googleusercontent.com/"]'))
        )
        image_url = [img.get_attribute('src') for img in image_element]
        
        if image_url:
            return image_url[0]
        else:
            return None
    
    except Exception as e:
        print(f"이미지를 찾을 수 없습니다: {query} - {e}")
        return None

with open(failed_query,'w') as fq:
    for index,row in culture_test.iterrows():
        shop_name = row['문화시설명']
        Id = row['주제분류']
        address_column = '주소'
        shop_column = '문화시설명'
        query = f"{row[address_column]} {row[shop_column]}"
        image_url = search_and_click(query)

        if image_url:
            try:
                response = requests.get(image_url)
                response.raise_for_status()
                img = Image.open(BytesIO(response.content))
                img.save(os.path.join(output_dir, f"{shop_name}.jpg"))
                print(f"Saved image for {row[address_column]} {shop_name}")
            except Exception as e:
                print(f"이미지 저장 오류: {e}")
        else:
            fq.write(f"Failed image): {query}\n")
print("Image crawling and saving completed.")
driver.quit()

Saved image for 서울 노원구 공릉동 107-51 육군박물관
Saved image for 서울특별시 중구 정동길 26 이화여고 100주년기념관
Saved image for 서울 종로구 북촌로5길 48 정독도서관 서울교육박물관
Saved image for 서울특별시 성북구 화랑로13길 60 동덕여자대학교 박물관
이미지를 찾을 수 없습니다: 서울특별시 용산구 서빙고로 17 용산센트럴파크 해링턴스퀘어 공공시설동 1,2층 용산도시기억전시관 - Message: 

Saved image for 서울 종로구 창경궁로 296-12 카톨릭대학교 가톨릭대학교 전례박물관
이미지를 찾을 수 없습니다: 서울 종로구 대학로 101 서울대학교병원 서울대학교 치의학대학원 치의학박물관 - Message: 

Saved image for 서울특별시 강동구 올림픽로 875 서울암사동유적
Saved image for 서울특별시 중구 소월로 91 안중근의사기념관
Saved image for 서울  종로구 새문안로 55 서울역사박물관
Saved image for 서울 강남구 도산대로 45길 20 도산안창호기념관 도산안창호 기념관
Saved image for 서울특별시 서초구 효령로72길 60 전기박물관
Saved image for 서울특별시 동대문구 약령중앙로 26 서울한방진흥센터
Saved image for 서울특별시 동대문구 경희대로 26 경희대학교 자연사박물관
Saved image for 서울특별시 서대문구 이화여대길 52 이화여자대학교 자연사박물관
Saved image for 서울특별시 서초구 바우뫼로7길 111 우면산관문사
Saved image for 서울특별시 중구 칠패로 5 서소문성지 역사박물관
Saved image for 서울특별시 동대문구 회기로 56 세종대왕기념관
Saved image for 서울특별시 용산구 청파로47길 99 숙명여자대학교 박물관
Saved image for 서울특별시 서초구 신흥안길 40-10 한국분재박물관
Saved image for 서울특별시 중구

In [44]:
"""
서울특별시 마포구 와우산로 121 에스제이비보이즈 - 삭제해야함
서울특별시 서초구 사임당로18길 52-2 정효아트홀 - 정효문화재단(이름 변경)
서울특별시 동대문구 안암로 86-1 크리에이터스 스페이스 - 삭제
서울특별시 영등포구 여의공원로 13 KBS홀 - 서울 영등포구 여의공원로 13 한국방송공사(주소 변경)
서울특별시 동작구 현충로 75 소태산홀 - 삭제
석촌호수 아뜰리에(중복) - 1개 삭제 
서울특별시 중구 소파로 142 드라마센터(동랑예술센터) - 서울 중구 소파로 138(주소 변경)
서울특별시 종로구 동숭길 122 동숭아트센터(폐업) - 삭제
서울특별시 중랑구 망우로 328 상봉시네마 - 삭제
서울특별시 성북구 동소문로20가길 28 대안공간 ○:PEN - 삭제
서울특별시 서초구 남부순환로323길 53 클라라 아트홀- 클라라아카데미홀(이름변경)
서울특별시 서대문구 연세로 50 연세대학교 - 삭제
서울특별시 송파구 올림픽로 240 롯데월드 가든스테이지 - 삭제
서울특별시 중구 새문안로 22 문화일보홀(현 청춘극장) - (현 청춘극장) -> 청춘극장(이름변경)
동숭무대 - 동숭무대소극장(이름변경)
-----------------------------------------------------
#######기타
서울특별시 마포구 매봉산로 31 tbs(서울특별시미디어재단) -> 서울특별시 미디어재단 티비에스(이름변경)
SPACE 1257 - 삭제
서울특별시 강남구 테헤란로87길 46 강남힐링센터(코엑스) - 삭제
서울특별시 중구 세종대로 110 서울시청 하늘광장 갤러리 - 하늘광장 갤러리만 쳐야함
서울특별시 송파구 백제고분로 399, 2층 에즈에스 아카데미 에즈에스아카데미 - 삭제
서울특별시 중랑구 송림길 172 중랑캠핑숲 - 서울 중랑구 망우로87길 110(주소변경)
서울특별시 서초구 반포대로 58 반포대로(띄어쓰기 있음 끝에) - 삭제
서울특별시 강서구 마곡동로 161 서울식물원
서울애니메이션센터(폐업?) - 삭제
서울 동작구 노량진동 47-2 동작구 문화관광 - 삭제
서울특별시 광진구 광나루로 571 구의문화재 - 삭제
충무로영상센터 오!재미동 - 충무로영상센터 오재미동(이름변경)
서울특별시 강북구 도봉로 76가길 55 (성신여대운정그린캠퍼스) 성신여자대학교 운정그린캠퍼스 대강당 - 삭제
서울특별시 종로구 송월길 52 서울시민대학 중부권 캠퍼스 - 삭제
서울특별시 중구 퇴계로 지하 431-1 신당역 유휴공간 - 삭제
서울특별시 동작구 사당로 143 총신대학교 백남조 기념홀 - 삭제
서울특별시 영등포구 양평로22마길 2 LES601 선유
서울특별시 중구 퇴계로20길 42 상상비즈아카데미
서울 종로구 세종로 76-14 의정부지 역사유적광장
서울특별시 종로구 자하문로10길 30 대우재단빌딩 ORBIS
서울특별시 서초구 강남대로2길 7 오소리 아트홀
서울특별시 동작구 흑석로 93-1 재단법인 동작문화재단
서울특별시 은평구 녹번동 5-29 맛동
서울특별시 도봉구 마들로11길 75 무중력지대 도봉(폐업)
서울특별시 도봉구 노해로63길 43 도봉문화재단
서울특별시 송파구 삼전로13길 22 아트잠실
서울 도봉구 창동 창동역 1번출구 광장
서울특별시 은평구 연서로48길 74 은평한옥마을어울림터
서울특별시 동대문구 경희대로 26 경희대학교
서울 노원구 공릉동 1-1 육군사관학교
서울생각마루(폐업?)
서울특별시 중구 장충단로 72 예지원
서울특별시 마포구 마포대로 89 서울특별시여성능력개발원
서울특별시 동대문구 천호대로 145 동대문구청 아트갤러리
서울특별시 영등포구 양산로 지하 200 아트온더무브
서울특별시 중구 만리동1가 1 서울로 7017
서울특별시 서초구 염곡말길 9 내곡느티나무쉼터
서울 동대문구 청계천로 517 서울시창작공간
서울특별시 서초구 올림픽대로 2085-14 세빛섬
서울특별시 성동구 자동차시장1길 64 장안평 자동차산업 종합정보센터
서울특별시 강서구 화곡로63길 96 등촌지역정보센터
서울특별시 금천구 시흥대로73길 70 금나래갤러리
서울특별시 종로구 북촌로 77-3 호경재
서울특별시 중랑구 신내로15길 189 중랑구민체육센터
서울특별시 동대문구 장안벚꽃로 67 동대문구민체육센터
--------------------------------------------------------
서울특별시 동작구 장승배기로16길 98 동작어린이도서관 - 동작영어마루도서관(이름변경)
서울특별시 구로구 구로동로26길 54 구로구립꿈나무어린이도서관 - 구로 꿈나무어린이도서관(이름변경)
서울특별시 노원구 공릉로51길 20, 지하 1층 문화살롱 5120 - 서울 노원구 공릉로51길 20(주소변경 - 지하1층 뺴기)
서울특별시 성동구 왕십리로11길 19 생활문화센터 - 서울숲아이꿈누리터(이름변경)
서울특별시 송파구 충민로 10 가든파이브 툴관 3층 E30 장지공방 - 삭제
서울특별시 양천구 남부순환로83길 54 건강힐링센터 - 건강힐링문화관 건강힐링센터(이름변경)
서울특별시 중랑구 면목로 238(면목동, 중랑구민회관 3층) 중랑문화원 - 서울 중랑구 면목로 238(주소변경)
주한영국문화원 - 주한영국문화원 영러너 시청어학원(이름변경)
서울특별시 서초구 강남대로 201 서초문화예술회관 2층 서초문화원 서초문화원 - 서울 서초구 강남대로 201(주소변경)
서울시 마포구 상암산로 1길 67 마포청소년문화의집 구립마포청소년문화의집 - 주소에서 마포청소년문화의집 빼기
더 소호 - 삭제
대림미술관(임시 휴업)
서울특별시 서초구 강남대로69길 8 브랜뉴디지털아트갤러리 - 삭제
서울특별시 중구 세종대로21길 33 조선일보미술관 (폐업) - 삭제
서울시립미술관 서소문본관 - 서울시립미술관(이름변경)
갤러리 룩스(임시 휴업) - 삭제
알비 (ARBY) - 서울 성동구 서울숲6길 8-1(주소변경), 알비(이름변경)
서울특별시 강남구 압구정로60길 13 시그니처 키친 스위트 청담 - 삭제
서울특별시 마포구 성산로 128 마포중앙도서관 갤러리 - 삭제 
세종아트갤러리 - 세종뮤지엄갤러리(이름변경)
공근혜갤러리 K.O.N.G. GALLERY - 공근혜갤러리(이름변경)
서울시립미술관 SeMA창고 - SeMA 창고(이름변경)
서울특별시 용산구 이태원로 256 스페이스 신선 - 삭제
서울 중구 서애로 33 5F 마프MAF - 서울 중구 서애로 33(주소변경)
미술관, 문화원, 문화예술회관, 도서관, 기타, 공연장
-> 박물관/기념관 해야함 
서울특별시 용산구 서빙고로 17 용산센트럴파크 해링턴스퀘어 공공시설동 1,2층 용산도시기억전시관
서울 종로구 세종대로 152 일민미술관건물 5층, 6층 신문박물관PRESSEUM - 주소에서 일민미술관건물 5층, 6층 이거 뺌
서울 성동구 용답동 250-7 서울하수도과학관 - 서울 성동구 자동차시장3길 64(주소변경)
서울 종로구 창덕궁길 94 원서노인정 창덕궁 돈화문 - 주소에서 원서노인정 뻄
서울  종로구 와룡동 110-360 ) 국립서울과학관 - 서울 종로구 창경궁로 215 국립어린이과학관(주소) 국립어린이과학관(시설명)
서울특별시 마포구 동교로15길 10 최규하 가옥 - 서울 마포구 서교동(주소변경)
서울특별시 종로구 혜화로5길 53 서울 명륜동 장면 가옥  - 서울 종로구 혜화로5길 49(주소변경), 장면가옥(이름변경)
서울 종로구 인사동길 35-4 인사동마루 본관 4~6층 뮤지엄김치간 - 서울 종로구 인사동길 35-4(주소변경)
서울특별시 중구 을지로 281 동대문역사문화공원 내 동대문운동장기념관 - 동대문역사문화공원동대문운동장기념관(이름변경)
서울특별시 광진구 능동로21길 37 세종대학교 박물관 - 서울 광진구 능동로 209(주소변경)
서울특별시 마포구 월드컵북로11길 20 전쟁과여성인권박물관 - 서울 마포구 월드컵북로11길 20 전쟁과여성인권박물관(주소변경)
서울 도봉구 쌍문동 산 250 둘리뮤지엄 - 서울 도봉구 시루봉로1길 6(주소변경)
서울특별시 종로구 세종대로 지하 175 세종충무공이야기 - 세종로 81-3(주소변경)
서울특별시 강남구 테헤란로 528 슈페리어타워 B2 세계골프역사박물관 세계골프역사박물관 - 서울 강남구 테헤란로 528(주소변경)
서울시 용산구 청파로47다길 27, 3층 (청파동2가 서현빌딩) 식민지역사박물관 - 서울 용산구 청파로47다길 27(주소변경)
서울특별시 용산구 청파로47길 100 숙명여자대학교 정영양자수박물관 - 서울 용산구 청파로47길 99 르네상스플라자 1F(주소변경)
서울특별시 강남구 테헤란로 528 슈페리어타워 지하2층 세계골프역사박물관 -중복(삭제)
서울 종로구 돈화문로 71 인산빌딩 떡박물관 - 인산빌딩 삭제
서울특별시 은평구 진관2로 57-23 서울금성당 샤머니즘박물관 - 삭제
"""

'\n서울특별시 종로구 종로33길 15 두산아트센터 스페이스111\n서울특별시 마포구 와우산로 121 에스제이비보이즈\n서울특별시 서초구 올림픽대로 2085-14 예빛섬\n서울특별시 서초구 사임당로18길 52-2 정효아트홀\n서울특별시 구로구 경인로20가길 38 오류아트홀\n서울특별시 종로구 대학로11길 41-8 동방뮤지컬컴퍼니\n서울특별시 동대문구 안암로 86-1 크리에이터스 스페이스\n서울 서초구 남부순환로317길 79 로데아트센터 로데아트센터\n소태산홀\n서울특별시 성북구 동소문로 24 공간222\n브람스앤리스트\n석촌호수 아뜰리에(중복)\n서울특별시 성북구 정릉로 77 국민대학교 예술관\n서울특별시 중구 소파로 142 드라마센터(동랑예술센터)\n서울특별시 관악구 신림로3길 35 관악아트홀\n서울특별시 서초구 명달로 104 모차르트홀\n서울특별시 종로구 동숭길 122 동숭아트센터(폐업)\n서울특별시 광진구 능동로 120 건국대학교 새천년대공연장\n서울 중구 정동길 3 경향신문사건물 경향아트힐\n서울특별시 중랑구 망우로 328 상봉시네마\n서울특별시 송파구 잠실로 124 서울놀이마당\n서울특별시 강북구 월계로 173 북서울꿈의숲아트센터\n서울특별시 성북구 동소문로20가길 28 대안공간 ○:PEN\n서울특별시 서초구 남부순환로323길 53 클라라 아트홀\n서울특별시 성동구 뚝섬로 273 서울숲야외무대\n서울특별시 강남구 학동로 163 하츠아트홀\n서울특별시 서초구 반포대로 20 라율아트홀\n서울특별시 종로구 이화장길 66 한예극장\n서울특별시 서대문구 연세로 50 연세대학교\n서울특별시 송파구 올림픽로 240 롯데월드 가든스테이지\n서울특별시 용산구 이태원로 213 스파찌오 루체\n서울특별시 서초구 서초대로40길 49 샤론아트홀\n서울특별시 중구 새문안로 22 문화일보홀(현 청춘극장)\n서울특별시 강남구 남부순환로378길 34-9 오유아트홀\n동숭무대\n서울특별시 마포구 매봉산로 31 tbs(서울특별시미디어재단)\nSPACE 1257\n서울특별시 강남구 테헤

### 폐업 시설 삭제

In [285]:
drop_values = ['에스제이비보이즈','크리에이터스 스페이스','소태산홀','동숭아트센터','상봉시네마','대안공간 ○:PEN','연세대학교',\
               '롯데월드 가든스테이지','장지공방','더 소호','브랜뉴디지털아트갤러리',\
               '조선일보미술관','갤러리 룩스','시그니처 키친 스위트 청담','마포중앙도서관 갤러리','스페이스 신선',\
               '서울금성당 샤머니즘박물관','장충체육관','롯데화랑(본점)','서울특별시 청년허브','아티스탄홀',\
               '양천구 해누리타운 아트홀','광화문아트홀','구로구립개봉어린이도서관','아트코리아랩 6층 아고라',\
               '공간 630','관악문화관','잠실종합운동장','이마트은평점','서울월드컵경기장','갤러리 가이아',
               '코엑스전시장','갤러리필로소피']
culture_kids = culture_kids.loc[~(culture_kids['문화시설명'].isin(drop_values))]

### 이름 변경

In [286]:
replace_dict ={'정효아트홀':'정효문화재단',
              '클라라 아트홀':'클라라아카데미홀',
              '문화일보홀(현 청춘극장)':'청춘극장',
              '동숭무대':'동숭무대소극장',
              '동작어린이도서관':'동작영어마루도서관',
              '구로구립꿈나무어린이도서관':'구로 꿈나무어린이도서관',
              '생활문화센터':'서울숲아이꿈누리터',
              '건강힐링센터':'건강힐링문화관 건강힐링센터',
              '주한영국문화원':'주한영국문화원 영러너 시청어학원',
              '서울시립미술관 서소문본관':'서울시립미술관',
              '알비 (ARBY)':'알비',
              '세종아트갤러리':'세종뮤지엄갤러리',
              '공근혜갤러리 K.O.N.G. GALLERY':'공근혜갤러리',
              '서울시립미술관 SeMA창고':'SeMA 창고',
              '국립서울과학관':'국립어린이과학관',
              '서울 명륜동 장면 가옥 ':'장면가옥',
              '동대문운동장기념관':'동대문역사문화공원동대문운동장기념관',
              '서울아트센터 도암홀':'서울아트센터',
               '수색동 주민센터(다목적실)':'수색동 주민센터',
               '강동어린이회관아이누리홀':'강동어린이회관 강동구육아종합지원센터',
               '세실극장':'국립정동극장 세실',
               'STUDIO301':'스튜디오301',
               '서초문화예술회관 (구 서초구민회관)':'서초문화예술회관',
               '서울성북미디어문화마루 꿈빛극장':'꿈빛극장',
               '아리랑 씨네센터':'아리랑시네센터',
               '한국의 집 민속극장':'한국의집',
               '북서울꿈의숲아트센터 퍼포먼스홀':'꿈의숲 아트센터 퍼포먼스홀',
               '강남구립못골한옥어린이도서관':'못골한옥어린이도서관',
               '개포도서관':'서울특별시교육청 개포도서관',
               '강서구립길꽃어린이도서관':'길꽃어린이도서관',
               '강남구립열린도서관':'열린도서관',
               '마포평생학습관 마포리움':'서울특별시교육청 마포평생학습관',
               '배봉산 숲속 도서관':'배봉산근린공원숲속도서관',
               '서초구립잠원도서관':'잠원도서관',
               '양천도서관':'서울특별시교육청 양천도서관',
               '서대문도서관':'서울특별시교육청서대문도서관',
               '정독도서관':'서울특별시교육청 정독도서관',
               '국립중앙도서관 실감서재':'국립중앙도서관',
               '이진아기념도서관':'서대문구립이진아기념도서관',
               '강서도서관':'서울특별시교육청 강서도서관',
               '동대문도서관':'서울특별시교육청 동대문도서관',
               '송파도서관':'서울특별시교육청 송파도서관',
               '고척도서관':'서울특별시교육청 고척도서관',
               '서울문화재단 대학로센터(예술청)':'서울문화재단 대학로센터',
               '아트스페이스 보안1942(통의동 보안여관)':'통의동 보안여관',
               '더 케이브 서울':'더 케이브',
               '서울장애예술창작센터':'서울문화재단 서울장애예술창작센터',
               '서울노인복지센터(탑골미술관)':'탑골미술관',
               '장디자인아트 JeanDesignArt':'장디자인아트',
               'CN갤러리 (충남문화예술 서울전시장)':'CN갤러리',
               '뮤지엄웨이브 (구 옛돌 박물관)':'뮤지엄웨이브',
               '국립현대미술관':'국립현대미술관 덕수궁',
               '인사아트프라자갤러리':'인사아트프라자 갤러리',
               '경춘선숲길 갤러리':'경춘선숲길갤러리',
               '갤러리 모나리자 산촌':'갤러리모나리자산촌',
               '토포하우스':'토포하우스 서울',
               '올리비아 박갤러리':'올리비아박 갤러리',
               '디지털 파빌리온':'디지털파빌리온',
               '밀알학교 세라믹 팔레스홀':'세라믹팔레스홀'
              }
culture_kids['문화시설명'] = culture_kids['문화시설명'].replace(replace_dict)
#culture_kids['문화시설명']

### 주소변경

In [287]:
replace_dict2 = {'서울특별시 영등포구 여의공원로 13':'서울 영등포구 여의공원로 13 한국방송공사',
                 '서울특별시 중구 소파로 142':'서울 중구 소파로 138',
                 '서울특별시 노원구 공릉로51길 20, 지하 1층':'서울 노원구 공릉로51길 20',
                 '서울특별시 중랑구 면목로 238(면목동, 중랑구민회관 3층)':'서울 중랑구 면목로 238',
                 '서울특별시 서초구 강남대로 201 서초문화예술회관 2층 서초문화원':'서울 서초구 강남대로 201',
                 '서울시 마포구 상암산로 1길 67 마포청소년문화의집':'서울시 마포구 상암산로 1길 67',
                 '서울 중구 서애로 33 5F':'서울 중구 서애로 33',
                 '서울특별시 용산구 서빙고로 17 용산센트럴파크 해링턴스퀘어 공공시설동 1,2층':'서울 용산구 서빙고로 17',
                 '서울 종로구 세종대로 152 일민미술관건물 5층, 6층':'서울 종로구 세종대로 152',
                 '서울 성동구 용답동 250-7':'서울 성동구 자동차시장3길 64',
                 '서울 종로구 창덕궁길 94 원서노인정':'서울 종로구 창덕궁길 94',
                 '서울  종로구 와룡동 110-360 )':'서울 종로구 창경궁로 215 국립어린이과학관',
                 '서울특별시 마포구 동교로15길 10':'서울 마포구 서교동',
                 '서울특별시 종로구 혜화로5길 53':'서울 종로구 혜화로5길 49',
                 '서울 종로구 인사동길 35-4 인사동마루 본관 4~6층':'서울 종로구 인사동길 35-4',
                 '서울특별시 중구 을지로 281 동대문역사문화공원 내':'서울특별시 중구 을지로 281',
                 '서울특별시 광진구 능동로21길 37':'서울 광진구 능동로 209',
                 '서울특별시 마포구 월드컵북로11길 20':'서울 마포구 월드컵북로11길 20 전쟁과여성인권박물관',
                 '서울 도봉구 쌍문동 산 250':'서울 도봉구 시루봉로1길 6',
                 '서울특별시 종로구 세종대로 지하 175':'세종로 81-3',
                 '서울특별시 강남구 테헤란로 528 슈페리어타워 B2 세계골프역사박물관':'서울 강남구 테헤란로 528',
                 '서울시 용산구 청파로47다길 27, 3층 (청파동2가 서현빌딩)':'서울 용산구 청파로47다길 27',
                 '서울특별시 용산구 청파로47길 100':'서울 용산구 청파로47길 99 르네상스플라자 1F',
                 '서울 종로구 돈화문로 71 인산빌딩':'서울 종로구 돈화문로 71',
                 '서울특별시 중구 손기정로 101-4':'서울 중구 손기정로 101 손기정공원',
                 '서울 서초구 방배동 981-13':'방배동 981-10',
                 '서울  강남구 대치동  영동대로 416, 코스모타워 3층':'서울 강남구 영동대로 416',
                 '서울 은평구 수색로 280 (수색동) 수색동 자치회관':'서울 은평구 수색로 280',
                 '서울특별시 강동구 성내로6길 16 (성내동)':'서울 강동구 성내로6길 16 강동어린이회관',
                 '서울특별시 강남구 역삼로7길 16, 3층':'서울특별시 강남구 역삼로7길 16',
                 '서울특별시 성동구 서울숲길 36, 3F Studio301':'서울 성동구 서울숲길 36',
                 '서울특별시 강서구 화곡로63길 65 B1':'서울특별시 강서구 화곡로63길 65',
                 '서울 강남구 봉은사로 524 코엑스인터콘티넨탈서울':'서울 강남구 봉은사로 524',
                 '서울 강남구 학동로 171 삼익악기빌딩 3층':'서울 강남구 학동로 171',
                 '서울특별시 강북구 월계로 173':'서울 강북구 월계로 173',
                 '서울특별시 종로구 동숭길 130-5 B1':'서울특별시 종로구 동숭길 130-5',
                 '서울특별시 종로구 대학로 120 (동숭동) 지하 1층':'서울특별시 종로구 대학로 120',
                 '서울시 송파구 오금로1 (신천동) 신천빗물펌프장 4~5층':'서울 송파구 오금로 1',
                 '서울특별시 노원구 중계로8길 20':'서울 노원구 중계로 106 불암도서관',
                 '서울시 은평구 증산로 17길 50 내를건너서숲으로도서관':'서울 은평구 증산로17길 50',
                 '서울 마포구 홍익로2길 16 (서교동, 마포평생학습관)':'서울 마포구 홍익로2길 16 마포평생학습관',
                 '서울특별시 영등포구 영중로 15':'서울 영등포구 문래로 196 영등포구 관광정보센터',
                 '서울특별시 종로구 율곡로5 (사간동 122-4)':'서울 종로구 율곡로 5',
                 '서울특별시 강남구 논현로 745':'서울 강남구 압구정로30길 63',
                 '서울특별시 성동구 서울숲6길 8-1 3층10':'서울특별시 성동구 서울숲6길 8-1',
                 '서울특별시 노원구 화랑로 608':'서울 노원구 공릉동 29-1',
                 '서울 종로구 삼청로 87 도올빌딩 2층':'서울 종로구 삼청로 87',
                 '서울시 강남구 일원로 90':'일원동 713'
    }
culture_kids['주소'] = culture_kids['주소'].replace(replace_dict2)

### 중복 제거

In [288]:
# 제거할 데이터들 인덱스로 제거
first_duplicate = culture_kids.loc[culture_kids['문화시설명'].isin(['세종뮤지엄갤러리','세계골프역사박물관','환기미술관','유니버설아트센터'])].drop_duplicates(subset=['문화시설명'],keep='last')
second_duplicate = culture_kids.loc[culture_kids['문화시설명'].isin(['석촌호수 아뜰리에','금천구립시흥도서관','신촌문화발전소','국립현대미술관 덕수궁'])].drop_duplicates(subset=['문화시설명'],keep='first')
drop_dup = pd.concat([first_duplicate,second_duplicate])
drop_dup
culture_kids = culture_kids.drop(drop_dup.index)

In [289]:
# culture_kids.loc[culture_kids['문화시설명'] == '환기미술관'] # keep ='first'
# culture_kids.loc[culture_kids['문화시설명'] == '유니버설아트센터'] # keep ='first'
# culture_kids.loc[culture_kids['문화시설명'] == '금천구립시흥도서관'] # keep = 'last'
# culture_kids.loc[culture_kids['문화시설명'] == '신촌문화발전소'] # keep = 'last'
# culture_kids.loc[culture_kids['문화시설명'] == '석촌호수 아뜰리에'] # keep='last'
# culture_kids.loc[culture_kids['문화시설명'] == '세종뮤지엄갤러리'] # keep = 'first'
# culture_kids.loc[culture_kids['문화시설명'] == '세계골프역사박물관'] # keep = 'first'

In [290]:
culture_kids['주제분류'] = culture_kids['주제분류'].replace(['공연장', '문화예술회관'], '공연장/문화예술회관')
culture_kids['주제분류'] = culture_kids['주제분류'].replace(['미술관','박물관/기념관'],'미술관/박물관/기념관')

In [291]:
#culture_kids.to_excel('preprocessed_culture_kids.xlsx',index=False)

### 키워드 추가

In [292]:
culture_kids = culture_kids.reset_index(drop=True)
culture_kids['대분류'] = '배우기'
culture_kids.rename(columns={'주제분류':'중분류'},inplace=True)

In [293]:
culture_kids['키워드'] = '추가'

In [295]:
classification1 = culture_kids['중분류'] == '공연장/문화예술회관'
classification1_key = ['서울공연', '서커스', '아트홀', '넓은 주차장', '친절함', '저렴함', \
                       '공연', '재밌음', '깨끗함', '뮤지컬', '콘서트', '볼거리 많음', '전시',\
                       '좋은 시설', '프로그램 많음', '쾌적함', '문화 프로그램', '체험',\
                       '대극장', '소극장', '실내', '단체 관람', '클래식']
culture_kids.loc[classification1,'키워드'] = ', '.join(classification1_key)

classification2 = culture_kids['중분류'] == '도서관'
classification2_key = ['예쁨', '아이', '어린이자료실', '친절함', '공공도서관', \
                      '한적함', '좋은 분위기', '넓은 주차장', '조용함', '책', '동화책', '영어원서', \
                      '독서 프로그램', '유아용 책', '어린이 열람실', '유모차 출입', '독서 이벤트']
culture_kids.loc[classification2,'키워드'] = ', '.join(classification2_key)

classification3 = culture_kids['중분류'] == '미술관/박물관/기념관'
classification3_key = ['미술관', '미술전시', '갤러리', '일러스트레이션', '굿즈', '작품',\
                       '공간 예쁨', '인상 깊음', '예약', '작가님', '그림', '예술', '힐링',\
                       '시각', '비주류 예술', '현대 예술', '박물관', '친절', '유익함', '알참',\
                       '문화재', '체험', '테마', '재밌음', '아이', '기념관' ,'art', '고전미술',\
                       '조각', '회화', '설치 미술', '예술가', '아트 프로그램', '미술 교육', \
                       '특별 전시', '예술 강연', '역사', '유적지', '전시관', '고대 문명', \
                       '공룡 화석', '투어', '어린이 박물관', '전통 문화', '기념 행사', '전시']
culture_kids.loc[classification3,'키워드'] = ', '.join(classification3_key)

classification4 = culture_kids['중분류'] == '문화원'
classification4_key = ['문화예술', '공공기관', '문화센터', '한적함', '깨끗함', '다양한 프로그램'\
                       , '배움', '강의', '강좌', '취미생활', '친절함', '저렴함', '문화생활'\
                       , '교양', '문화원', '전통문화', '상담', '강사님', '배울거리'\
                       , '어린이 강좌', '문화행사', '문화체럼', '문화축제', '문화교실'\
                       , '다문화 프로그램', '어린이 문화원', '어린이 체험', '어린이 공연']
culture_kids.loc[classification4,'키워드'] = ', '.join(classification4_key)

In [282]:
#culture_kids.to_excel('preprocessed_culture_kids.xlsx',index=False)